In [1]:
import json
import time
import urllib
import requests
from typing import *
from string import Template
from tqdm.notebook import tqdm

# Constants

In [2]:
API_SEARCH_QUERY = '("kebakaran hutan" or "kabut asap") min_retweets:10 lang:id'
API_MAX_SCROLL   = 100  # max scroll (pagination)
API_MAX_DEPTH    = 20   # max nested retweets depth

API_BASE_URL = "https://x.com/i/api/graphql"

API_SEARCH_TIMELINE_URL             = API_BASE_URL + "/bshMIjqDk8LTXTq4w91WKw/SearchTimeline"
API_SEARCH_TIMELINE_VARIABLES_PARAM = {
    "rawQuery"             : "${query}",
    "cursor"               : "${cursor}",
    "count"                : 20,
    "querySource"          : "typed_query",
    "product"              : "Latest",
    "withGrokTranslatedBio": False,
}

API_TWEET_DETAIL_URL             = API_BASE_URL + "/6QzqakNMdh_YzBAR9SYPkQ/TweetDetail"
API_TWEET_DETAIL_VARIABLES_PARAM = {
    "focalTweetId"                          : "${tweet_id}",
    "cursor"                                : "${cursor}",
    "referrer"                              : "search",
    "with_rux_injections"                   : False,
    "rankingMode"                           : "Relevance",
    "includePromotedContent"                : False,
    "withCommunity"                         : False,
    "withQuickPromoteEligibilityTweetFields": False,
    "withBirdwatchNotes"                    : False,
    "withVoice"                             : False,
}

API_FEATURES_PARAM = {
    "rweb_video_screen_enabled"                                              : False,
    "profile_label_improvements_pcf_label_in_post_enabled"                   : True,
    "responsive_web_profile_redirect_enabled"                                : False,
    "rweb_tipjar_consumption_enabled"                                        : True,
    "verified_phone_label_enabled"                                           : False,
    "creator_subscriptions_tweet_preview_api_enabled"                        : True,
    "responsive_web_graphql_timeline_navigation_enabled"                     : True,
    "responsive_web_graphql_skip_user_profile_image_extensions_enabled"      : False,
    "premium_content_api_read_enabled"                                       : False,
    "communities_web_enable_tweet_community_results_fetch"                   : True,
    "c9s_tweet_anatomy_moderator_badge_enabled"                              : True,
    "responsive_web_grok_analyze_button_fetch_trends_enabled"                : False,
    "responsive_web_grok_analyze_post_followups_enabled"                     : True,
    "responsive_web_jetfuel_frame"                                           : True,
    "responsive_web_grok_share_attachment_enabled"                           : True,
    "articles_preview_enabled"                                               : True,
    "responsive_web_edit_tweet_api_enabled"                                  : True,
    "graphql_is_translatable_rweb_tweet_is_translatable_enabled"             : True,
    "view_counts_everywhere_api_enabled"                                     : True,
    "longform_notetweets_consumption_enabled"                                : True,
    "responsive_web_twitter_article_tweet_consumption_enabled"               : True,
    "tweet_awards_web_tipping_enabled"                                       : False,
    "responsive_web_grok_show_grok_translated_post"                          : False,
    "responsive_web_grok_analysis_button_from_backend"                       : True,
    "creator_subscriptions_quote_tweet_preview_enabled"                      : False,
    "freedom_of_speech_not_reach_fetch_enabled"                              : True,
    "standardized_nudges_misinfo"                                            : True,
    "tweet_with_visibility_results_prefer_gql_limited_actions_policy_enabled": True,
    "longform_notetweets_rich_text_read_enabled"                             : True,
    "longform_notetweets_inline_media_enabled"                               : True,
    "responsive_web_grok_image_annotation_enabled"                           : True,
    "responsive_web_grok_imagine_annotation_enabled"                         : True,
    "responsive_web_grok_community_note_auto_translation_is_enabled"         : False,
    "responsive_web_enhance_cards_enabled"                                   : False,
}

# User many headers to avoid too many requests
API_LIST_HEADERS = [
    {
        "accept"                   : "*/*",
        "accept-language"          : "en-GB,en;q=0.9",
        "authorization"            : "Bearer AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
        "content-type"             : "application/json",
        "priority"                 : "u=1, i",
        "referer"                  : "https://x.com/search?q=(%22kebakaran%20hutan%22%20or%20%22kabut%20asap%22)%20min_retweets%3A10%20lang%3Aid&src=typed_query&f=live",
        "sec-ch-ua"                : '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        "sec-ch-ua-mobile"         : "?0",
        "sec-ch-ua-platform"       : '"macOS"',
        "sec-fetch-dest"           : "empty",
        "sec-fetch-mode"           : "cors",
        "sec-fetch-site"           : "same-origin",
        "user-agent"               : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
        "x-client-transaction-id"  : "gSIXQdWgCBSuND0GioxtQls+vYWR4rQi/yLH97+z3VSblN5i7hFwu0sVRp+YnVbOUZp3ZoWqwedRhNUrgebIl7+YmPHsgg",
        "x-csrf-token"             : "812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c",
        "x-twitter-active-user"    : "yes",
        "x-twitter-auth-type"      : "OAuth2Session",
        "x-twitter-client-language": "en",
        "x-xp-forwarded-for"       : "834df0c98ee4b8019c07dddeaf1378e449d6fb7c77bcce87182cee5ee225567bb64f1971f16d848bec9a38f236fd4fedcfd9b1fc55d1927cbcfa895e9a001425227cc61e9453e0fc70fd581aaf154b8c3ab341aa359f761d4fd4109dc730696eba10842b41fc12479b7c83582aead9ca6a4467a49ea3b5cd2a8e403ebed26e7bb319db6f5ea921ff8256c1c84bced018193a2f978b7cbd2eb21e907fdf9b9371d3eeaf88c26c26589cddac3ad0c6f99c63ac82011fcc3a6856e14886e43ce5874fd6153732a82b671124d0ac5a978150694d835248a79c2c8161aa464499c53d94fefa823947d605e59e8597cda99783e8ab9b7edcdedbc3f2f1374a11aa554997",
        "Cookie"                   : 'guest_id_marketing=v1%3A176523474814329995; guest_id_ads=v1%3A176523474814329995; guest_id=v1%3A176523474814329995; personalization_id="v1_xw03l5DWX2lxhaNR0F3B+g=="; gt=1998165475646455871; __cuid=32f1b2a198814f3892df122eb69af73b; g_state={"i_l":0,"i_ll":1765234768568,"i_b":"JlM/61x2AN5yK1rjGldIsiZBXDU8/YtTFzASntzP0LA"}; kdt=rLDiLi5arY92A1ZPrtq0L28MwL8L0uus7Fpp1jo5; auth_token=c2d158295830b988fe8479ca7e077d5ba9d2a9ca; ct0=812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c; att=1-qcZ3XxyTticJRtFKzY4v9sSiIqBXfqfHd1cPa8kx; lang=en; twid=u%3D1975584269155377155; __cf_bm=4s.jAM61aVQrMYUItVSo.c5WA1Uk4HwLVuZu.v29hAM-1765235083.125108-1.0.1.1-jnhumMdTSRkWYN4t2829spJ4g9gjLu3_3s6n4eJX6IkmGLq2DPWQ8LNQG7baMKCoMntxf0KZmmvAVmIC98A49S3jodcmduJe0T6w9zepzUD3fCU.gj3yvwaaUxBM8fuu; __cf_bm=kwliB7cKYN.5e.Trf.hT9Cct9rIVHlQeUbAkKFLaAa4-1765334611.407966-1.0.1.1-36yJtKnuU1B8QUUOadCeQiHbFLpXonXj87LDdHMbdBU1L13I6KcYSiOb5J9CKBnSTPFU0m22GmUnkJQZbz8SDuWTKPxk3YpI19y6uJiiqty439tc8P1gGFcw3o94vYwf; guest_id=v1%3A175929449319161090; guest_id_ads=v1%3A175929449319161090; guest_id_marketing=v1%3A175929449319161090; personalization_id="v1_OGejUgP0SkhENMP/lWNnPw=="',
    },
    {
        "accept"                   : "*/*",
        "accept-language"          : "en-GB,en;q=0.9",
        "authorization"            : "Bearer AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
        "content-type"             : "application/json",
        "priority"                 : "u=1, i",
        "referer"                  : "https://x.com/search?q=(%22kebakaran%20hutan%22%20or%20%22kabut%20asap%22)%20min_retweets%3A10%20lang%3Aid&src=recent_search_click&f=live",
        "sec-ch-ua"                : '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        "sec-ch-ua-mobile"         : "?0",
        "sec-ch-ua-platform"       : '"macOS"',
        "sec-fetch-dest"           : "empty",
        "sec-fetch-mode"           : "cors",
        "sec-fetch-site"           : "same-origin",
        "user-agent"               : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
        "x-client-transaction-id"  : "q8y/r8DtK/qSfI2NOWKpgwLLRTDDCJnqo/eOfYPzMWl2qemfRFAWmvTqDU/pnYUNP4bSQq/mBwqusUrlLc6HUcDlHx4GqA",
        "x-csrf-token"             : "812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c",
        "x-twitter-active-user"    : "yes",
        "x-twitter-auth-type"      : "OAuth2Session",
        "x-twitter-client-language": "en",
        "x-xp-forwarded-for"       : "988c2c37159e4216ee9031324117716d950cdb2e4806c24892dbb0afd5a223c7347f2dea6e28583ef7ebd3931168ca20921fdfeed12f33432489546b01d74f10b3ee2cf9f110482d3a4c2b4e833cda5849dbd4fe06cd181473975fd1df17948a2e180967000bbbcfd571a87bd6665d0c3616c6aa6123ebbffef1cf481a1ba5fece75e005b4e81da882e9d8f7193394a705f44a3bfc8bd97571736baf7e2ef682d88764aa77b160253f2bc5fb66ae865b56ec1ce1cf861e58b36e33035659854b7740dd4a5e208fce429f735e6fae551dc669f1edc54520038c16c461e2425356f960ac6ecc77f4b507c6055df8f830ed073ca11c7c49449557c222a28e79909891",
        "Cookie"                   : 'guest_id_marketing=v1%3A176523474814329995; guest_id_ads=v1%3A176523474814329995; guest_id=v1%3A176523474814329995; personalization_id="v1_xw03l5DWX2lxhaNR0F3B+g=="; __cuid=32f1b2a198814f3892df122eb69af73b; g_state={"i_l":0,"i_ll":1765234768568,"i_b":"JlM/61x2AN5yK1rjGldIsiZBXDU8/YtTFzASntzP0LA"}; kdt=rLDiLi5arY92A1ZPrtq0L28MwL8L0uus7Fpp1jo5; auth_token=c2d158295830b988fe8479ca7e077d5ba9d2a9ca; ct0=812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c; lang=en; twid=u%3D1975584269155377155; __cf_bm=B_HE.7mKFIOM09sVc88.z1f7flxQ1SQ6FL24lCm_IyA-1765334172.7441645-1.0.1.1-6e8FFk5jUa9350eh6tfDgSWu0TON3vLes5zpkY_RsHPUWsHErjQ2eK2hjiYqTJu0S3m4UU0c_6L3gBt0MaMAvcEf1pvW6yHI_Do1j0gyAnojNKkle8netYVh_INpy413; __cf_bm=X_HI8wd.8p5doXjTNoDPFH7UqGzy3vQLfq90hFN.rHQ-1765334605.3624809-1.0.1.1-qOZMxEA3tsy5OpfcEL5fTX44KmI3jwdHMtmqmkTq6eSNuLIHEym27_Rxt879EHpp1.DeKP8XPbSV8cJ5WlFgESpaAAUamSskuMD6w7Pp5NZZzZbszS..6iPisFC551W1; guest_id=v1%3A175929449319161090; guest_id_ads=v1%3A175929449319161090; guest_id_marketing=v1%3A175929449319161090; personalization_id="v1_OGejUgP0SkhENMP/lWNnPw=="',
    },
    {
        "accept"                   : "*/*",
        "accept-language"          : "en-GB,en;q=0.9",
        "authorization"            : "Bearer AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
        "content-type"             : "application/json",
        "priority"                 : "u=1, i",
        "referer"                  : "https://x.com/search?q=(%22kebakaran%20hutan%22%20or%20%22kabut%20asap%22)%20min_retweets%3A10%20lang%3Aid&src=recent_search_click&f=live",
        "sec-ch-ua"                : '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        "sec-ch-ua-mobile"         : "?0",
        "sec-ch-ua-platform"       : '"macOS"',
        "sec-fetch-dest"           : "empty",
        "sec-fetch-mode"           : "cors",
        "sec-fetch-site"           : "same-origin",
        "user-agent"               : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
        "x-client-transaction-id"  : "2h51yLFBOjQc5XCNxu7f5+FwXIzesZojZU6GQSxDaZIhriUrLB0a7fg7kAQ4npvpTaWgM97Zsh+dg6/7QMsth3edXOnE2Q",
        "x-csrf-token"             : "812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c",
        "x-twitter-active-user"    : "yes",
        "x-twitter-auth-type"      : "OAuth2Session",
        "x-twitter-client-language": "en",
        "x-xp-forwarded-for"       : "c8cb7fbdb0749d40e336fcb43d030943d7da14f4614ca5a814cc6abe7ac6804f1d0dba7caa070cc5e3ae3367c8d73ce9365bf7fe6cb1eb99b2af87b5cf980f4d2b290e8b304a5fc4c20a300b8ee35c9a53fd7ae8f2b6fd0b3131b0554bd2f4bbf02cc3768b3f29bd9c4fd2fb95684a78b39ca7cff48661359f7dc45d84983ec3de88b78d5b3a6d073633a94cdd977a9022eed5cd54783bc86ae5f58b59253fd500d0f92ceb9972d06ec03c80af9aa3f542021d43fdef28dc6fca26a662d16e321b3587945d0a2059815d5a853d2a7d5538cef3b882ff1fdb92df818f00e93f6fc8e2ca646145ff1ef378040bba40fd1fe9ebe29fe2012be882f0533f4ef129c5fd",
        "Cookie"                   : 'guest_id_marketing=v1%3A176523474814329995; guest_id_ads=v1%3A176523474814329995; guest_id=v1%3A176523474814329995; personalization_id="v1_xw03l5DWX2lxhaNR0F3B+g=="; __cuid=32f1b2a198814f3892df122eb69af73b; g_state={"i_l":0,"i_ll":1765234768568,"i_b":"JlM/61x2AN5yK1rjGldIsiZBXDU8/YtTFzASntzP0LA"}; kdt=rLDiLi5arY92A1ZPrtq0L28MwL8L0uus7Fpp1jo5; auth_token=c2d158295830b988fe8479ca7e077d5ba9d2a9ca; ct0=812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c; lang=en; twid=u%3D1975584269155377155; __cf_bm=1z5.EuDTXzMR92PUcVZxkzi52TW5mGE6puCx3kqOjUs-1765334510.9110289-1.0.1.1-8W_Qs.NzyAM1f1p6jxa0pdgihg5SDsgM8V8X8oQjAhqTQ1zd71PX0bNNCYWeoe1dYhq8xO_n0sPplnSlzixl6vYRDvkJ3eivtUSj_IiLhE7C37KCp4qSdkcrMU_4a4qL; __cf_bm=ZGwDxZrM9mCV9OHr8ppsaHLtTsqe..XkHZHaTCSUCGI-1765334607.7784972-1.0.1.1-DGmjrooskiEacvclCy9zfv9XnKcGoKBRp5JuB5k7f54uYu.TOnL1X.s3L3m8T70JFXfZgx_6BG_0MvLLrnokFCT1MraGBsgsMAEGe7pfVAqaCZmwWVn1y4_3ftnEhNVH; guest_id=v1%3A175929449319161090; guest_id_ads=v1%3A175929449319161090; guest_id_marketing=v1%3A175929449319161090; personalization_id="v1_OGejUgP0SkhENMP/lWNnPw=="',
    },
    {
        "accept"                   : "*/*",
        "accept-language"          : "en-GB,en;q=0.9",
        "authorization"            : "Bearer AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
        "content-type"             : "application/json",
        "priority"                 : "u=1, i",
        "referer"                  : "https://x.com/search?q=(%22kebakaran%20hutan%22%20or%20%22kabut%20asap%22)%20min_retweets%3A10%20lang%3Aid&src=recent_search_click&f=live",
        "sec-ch-ua"                : '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        "sec-ch-ua-mobile"         : "?0",
        "sec-ch-ua-platform"       : '"macOS"',
        "sec-fetch-dest"           : "empty",
        "sec-fetch-mode"           : "cors",
        "sec-fetch-site"           : "same-origin",
        "user-agent"               : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
        "x-client-transaction-id"  : "1pHcYpTFzH4qvVg2+l3o5cXQvwtElyIpAoLwGShq8DzznI6aH9G/X9FuPdEdTHmwqfWtP9Ihncx8Y8FDC2RWAAguJrU61Q",
        "x-csrf-token"             : "812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c",
        "x-twitter-active-user"    : "yes",
        "x-twitter-auth-type"      : "OAuth2Session",
        "x-twitter-client-language": "en",
        "x-xp-forwarded-for"       : "25a07a964a879eb9e84f92455d8152ce3a7ca372908d906a3de2b6028991ec22ecb2791b5b2de079076a181657b0d4ca781b2af613a799fd8778485ce5bdca0093d8c7aefe75f734b8c3c127d99dbd6acb9114c92e51fc772425d39dc7e27e0979a21ca173ade37a4c226f7c831944c280bcd8a918240ad00b7491ccdd25a234e752dc7736bd42eaf8d492fc31f5762f44563f58ba5bb4aa7832b9ca24a26fcdee85239b9dd53d6def97f03e5530606414ec360b8568ea387f919e80b0dad31f5b3b34e139bb689f8ce600a4636e91bfe8bfbd0cec23cd074dcffd671e2170b36c679e6ef179816b6045a6a9e1ac12e1a86ca92e660c8d5857fd3fbc51458874",
        "Cookie"                   : 'guest_id_marketing=v1%3A176523474814329995; guest_id_ads=v1%3A176523474814329995; guest_id=v1%3A176523474814329995; personalization_id="v1_xw03l5DWX2lxhaNR0F3B+g=="; __cuid=32f1b2a198814f3892df122eb69af73b; g_state={"i_l":0,"i_ll":1765234768568,"i_b":"JlM/61x2AN5yK1rjGldIsiZBXDU8/YtTFzASntzP0LA"}; kdt=rLDiLi5arY92A1ZPrtq0L28MwL8L0uus7Fpp1jo5; auth_token=c2d158295830b988fe8479ca7e077d5ba9d2a9ca; ct0=812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c; lang=en; twid=u%3D1975584269155377155; __cf_bm=qZYozKpEdFfb53lk51km7hbFIRqZvre_nHNcyuMJGsA-1765334674.5010958-1.0.1.1-v_L0VXYND.ZFudrFseYYUK7FgqtDspT7HkEhlEYSOWfklFqejaO68RnJy_GnKI_g1vOID4RSSpyH4B8yw5nwoiaoavVjh7zXQVOxk_BSmrPBS.xWwmvyaeon6LXxvWHW; __cf_bm=kwliB7cKYN.5e.Trf.hT9Cct9rIVHlQeUbAkKFLaAa4-1765334611.407966-1.0.1.1-36yJtKnuU1B8QUUOadCeQiHbFLpXonXj87LDdHMbdBU1L13I6KcYSiOb5J9CKBnSTPFU0m22GmUnkJQZbz8SDuWTKPxk3YpI19y6uJiiqty439tc8P1gGFcw3o94vYwf; guest_id=v1%3A175929449319161090; guest_id_ads=v1%3A175929449319161090; guest_id_marketing=v1%3A175929449319161090; personalization_id="v1_OGejUgP0SkhENMP/lWNnPw=="',
    },
    {
        "accept"                   : "*/*",
        "accept-language"          : "en-GB,en;q=0.9",
        "authorization"            : "Bearer AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
        "content-type"             : "application/json",
        "priority"                 : "u=1, i",
        "referer"                  : "https://x.com/search?q=(%22kebakaran%20hutan%22%20or%20%22kabut%20asap%22)%20min_retweets%3A10%20lang%3Aid&src=recent_search_click&f=live",
        "sec-ch-ua"                : '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        "sec-ch-ua-mobile"         : "?0",
        "sec-ch-ua-platform"       : '"macOS"',
        "sec-fetch-dest"           : "empty",
        "sec-fetch-mode"           : "cors",
        "sec-fetch-site"           : "same-origin",
        "user-agent"               : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
        "x-client-transaction-id"  : "dKEgxWsSCs9bkpoGGSjEIGgVn4zR2cjZiJk8gh31DILFjVqiFQ7LublCBuJ+AYPVpJ4PnXAZ6S4MTIbApkr10+ZlIfrSdw",
        "x-csrf-token"             : "812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c",
        "x-twitter-active-user"    : "yes",
        "x-twitter-auth-type"      : "OAuth2Session",
        "x-twitter-client-language": "en",
        "x-xp-forwarded-for"       : "becf58d1dfc29fae8c63ce84928d7c4a52c793749cf6e7b441a1a26812d473efbd141d66391d593e5ea1936af6e3cb645a72629d8de4b85963be3f09a7582caabe91e94a7803ce62c23f72941dd6158a624b7904e1438bba4a1b7ac59152d7e80d9b12f16b0d6b2ab17e7b0a2d976ae3ba4d5931a6445a3381aa8d1f51cbc569084a408b378e3ba0b0d4a1da93ca1416777de18cbf88cae6142eb462c87ee135119948458d2f1ddd17b25dceb9c4fbcd80794e5051701a1a7b65496b1a9ce5aeb413f7db1e5ace4090decd6ba75d2426a02b1fb8371655d8d12fb7884b72f36fb451622c4c773a22b8e02a4801db72d4571d1f16ecc13ad5ffb49b29631d4a6487",
        "Cookie"                   : 'guest_id_marketing=v1%3A176523474814329995; guest_id_ads=v1%3A176523474814329995; guest_id=v1%3A176523474814329995; personalization_id="v1_xw03l5DWX2lxhaNR0F3B+g=="; __cuid=32f1b2a198814f3892df122eb69af73b; g_state={"i_l":0,"i_ll":1765234768568,"i_b":"JlM/61x2AN5yK1rjGldIsiZBXDU8/YtTFzASntzP0LA"}; kdt=rLDiLi5arY92A1ZPrtq0L28MwL8L0uus7Fpp1jo5; auth_token=c2d158295830b988fe8479ca7e077d5ba9d2a9ca; ct0=812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c; lang=en; twid=u%3D1975584269155377155; __cf_bm=wzjYknCzP_7VGmx92lrRoQjBOz0L0Dg22eexUpD6FA8-1765334874.743889-1.0.1.1-hR_OG5mqE56suwZS.a_nwP0Bea3AMNkQiiF.19Kg14at0UT5ZwiCkQPIiWcRRd6zvDfnSnWshsWZitBi_o9QzCdBeETbvCx26.7ERE27tD2XyN1btFNjSEMsETZayJUe; __cf_bm=kwliB7cKYN.5e.Trf.hT9Cct9rIVHlQeUbAkKFLaAa4-1765334611.407966-1.0.1.1-36yJtKnuU1B8QUUOadCeQiHbFLpXonXj87LDdHMbdBU1L13I6KcYSiOb5J9CKBnSTPFU0m22GmUnkJQZbz8SDuWTKPxk3YpI19y6uJiiqty439tc8P1gGFcw3o94vYwf; guest_id=v1%3A175929449319161090; guest_id_ads=v1%3A175929449319161090; guest_id_marketing=v1%3A175929449319161090; personalization_id="v1_OGejUgP0SkhENMP/lWNnPw=="',
    },
    {
        "accept"                   : "*/*",
        "accept-language"          : "en-GB,en;q=0.9",
        "authorization"            : "Bearer AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
        "content-type"             : "application/json",
        "priority"                 : "u=1, i",
        "referer"                  : "https://x.com/search?q=(%22kebakaran%20hutan%22%20or%20%22kabut%20asap%22)%20min_retweets%3A10%20lang%3Aid&src=recent_search_click&f=live",
        "sec-ch-ua"                : '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        "sec-ch-ua-mobile"         : "?0",
        "sec-ch-ua-platform"       : '"macOS"',
        "sec-fetch-dest"           : "empty",
        "sec-fetch-mode"           : "cors",
        "sec-fetch-site"           : "same-origin",
        "user-agent"               : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
        "x-client-transaction-id"  : "jPTtFy1g4ALx9P0wrhjZ29dFZOkeljmaRfLxATNYYuYJEIbf8gkN2iGErRijPPYCaLfwZYiOnmkGMPf0q6ZnK8Mt5Fltjw",
        "x-csrf-token"             : "812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c",
        "x-twitter-active-user"    : "yes",
        "x-twitter-auth-type"      : "OAuth2Session",
        "x-twitter-client-language": "en",
        "x-xp-forwarded-for"       : "cf2ac959ab22561728297b8f6e726cd184cea9013359b108099ef5b172322422fcbbcea1921aca06350e22884b0034b57a8ca41c5273ab275e9dec4c4fb0ff12ab32e3dbeeb0cf8475ecba477bdf8ff9d4a450b74cd85d33f35df6163808043db8e23c2cbd92757f88c2f5d5395dbed04407a267d24ae49ba0f333211db5207abac0c9f03f5a27c2f19c619c39c91d37eb0fb93db12ee5da0b550a835f9aa5ffe9ca4b3a5b6f19d4de811d53667b61bc3e55c0014d11e52a597a0b9383b36989439f7260e7b1b7bc4e244307b5e3f781cd79fa09d01aa0f42c557d585a00884c31fb40b05e95ca98a3e030b6cb9ff9e3c4954ec3ca049d41d510ee432cb0ae5e69",
        "Cookie"                   : 'guest_id_marketing=v1%3A176523474814329995; guest_id_ads=v1%3A176523474814329995; guest_id=v1%3A176523474814329995; personalization_id="v1_xw03l5DWX2lxhaNR0F3B+g=="; __cuid=32f1b2a198814f3892df122eb69af73b; g_state={"i_l":0,"i_ll":1765234768568,"i_b":"JlM/61x2AN5yK1rjGldIsiZBXDU8/YtTFzASntzP0LA"}; kdt=rLDiLi5arY92A1ZPrtq0L28MwL8L0uus7Fpp1jo5; auth_token=c2d158295830b988fe8479ca7e077d5ba9d2a9ca; ct0=812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c; lang=en; twid=u%3D1975584269155377155; __cf_bm=gghGjHeXKByBiFs5L.d4h89SCG0JtMgihL66XLjT45c-1765334955.239195-1.0.1.1-psSmvXEL._mWAXailpEhVT7raHZHSliLsncA0gGDWDaVkiiPAuPu089jxFObopBDur7YMDkpHjN9Wk3KDDW0j2hN4Ry2Pw1jMHho9x96GEot2cQ3tivFCvsM6pV7_w40; __cf_bm=kwliB7cKYN.5e.Trf.hT9Cct9rIVHlQeUbAkKFLaAa4-1765334611.407966-1.0.1.1-36yJtKnuU1B8QUUOadCeQiHbFLpXonXj87LDdHMbdBU1L13I6KcYSiOb5J9CKBnSTPFU0m22GmUnkJQZbz8SDuWTKPxk3YpI19y6uJiiqty439tc8P1gGFcw3o94vYwf; guest_id=v1%3A175929449319161090; guest_id_ads=v1%3A175929449319161090; guest_id_marketing=v1%3A175929449319161090; personalization_id="v1_OGejUgP0SkhENMP/lWNnPw=="',
    },
    {
        "accept"                   : "*/*",
        "accept-language"          : "en-GB,en;q=0.9",
        "authorization"            : "Bearer AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
        "content-type"             : "application/json",
        "priority"                 : "u=1, i",
        "referer"                  : "https://x.com/search?q=(%22kebakaran%20hutan%22%20or%20%22kabut%20asap%22)%20min_retweets%3A10%20lang%3Aid&src=recent_search_click&f=live",
        "sec-ch-ua"                : '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        "sec-ch-ua-mobile"         : "?0",
        "sec-ch-ua-platform"       : '"macOS"',
        "sec-fetch-dest"           : "empty",
        "sec-fetch-mode"           : "cors",
        "sec-fetch-site"           : "same-origin",
        "user-agent"               : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
        "x-client-transaction-id"  : "wlYPQLxoVy04KQMkRs2g8hV6L8c7cl5kjgFOhhX9k3R9NYRT3bZ3kSDbs7tR/bquSaW+K8ZSqYy60eks5CSFbi0ByVnWwQ",
        "x-csrf-token"             : "812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c",
        "x-twitter-active-user"    : "yes",
        "x-twitter-auth-type"      : "OAuth2Session",
        "x-twitter-client-language": "en",
        "x-xp-forwarded-for"       : "100b129be0f0f25c7497705ab8fef66816c59ff38e799bb96e3553947094c19df8306a941182d5f9a1dd7331c0accaf8d55f8afc7ce74707baeb3bf0f3fa9d9acc616b499796e6f2c0ff3b37a61845164b5787ab0c5801873ffdc87b48e949e46e85b17139515f954ccc32294c47a5e740a3b24405dfbaeabd8d6173de08b1f13351ef14ceab6cd76ec7931923034d2c1d7fff4ab96a83e5d38b4d8e05d175981d27120396123e58c85e48b2f8e9751046067b0580908c9e7fe875a172818fc2db9e517a7b9e5ef62c8fdd57c66d814242d26d53b7bee65a19fd015a5a4f2cbcd98f63fe771ac9c57ce01ba86920f337125fd1b85e04dd2487be2297f4ae65c5f8",
        "Cookie"                   : 'guest_id_marketing=v1%3A176523474814329995; guest_id_ads=v1%3A176523474814329995; guest_id=v1%3A176523474814329995; personalization_id="v1_xw03l5DWX2lxhaNR0F3B+g=="; __cuid=32f1b2a198814f3892df122eb69af73b; g_state={"i_l":0,"i_ll":1765234768568,"i_b":"JlM/61x2AN5yK1rjGldIsiZBXDU8/YtTFzASntzP0LA"}; kdt=rLDiLi5arY92A1ZPrtq0L28MwL8L0uus7Fpp1jo5; auth_token=c2d158295830b988fe8479ca7e077d5ba9d2a9ca; ct0=812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c; lang=en; twid=u%3D1975584269155377155; __cf_bm=gyZbTOIhwSgNSlReKOPQoYEbD7qheaMoBGY.ABSfNTc-1765334999.7894492-1.0.1.1-eFig_eQbP0ZBf_afHvp6zRokJY0RyTSvnU0hQRehDNxk8my9WkVFT8jAhN72y0stN8BvY4djN8ZoLj2bmC18Rge0_QhJIxm.rqza3briPungDq9vin_iffrUSWVaGM4F; __cf_bm=kwliB7cKYN.5e.Trf.hT9Cct9rIVHlQeUbAkKFLaAa4-1765334611.407966-1.0.1.1-36yJtKnuU1B8QUUOadCeQiHbFLpXonXj87LDdHMbdBU1L13I6KcYSiOb5J9CKBnSTPFU0m22GmUnkJQZbz8SDuWTKPxk3YpI19y6uJiiqty439tc8P1gGFcw3o94vYwf; guest_id=v1%3A175929449319161090; guest_id_ads=v1%3A175929449319161090; guest_id_marketing=v1%3A175929449319161090; personalization_id="v1_OGejUgP0SkhENMP/lWNnPw=="',
    },
    {
        "accept"                   : "*/*",
        "accept-language"          : "en-GB,en;q=0.9",
        "authorization"            : "Bearer AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
        "content-type"             : "application/json",
        "priority"                 : "u=1, i",
        "referer"                  : "https://x.com/search?q=(%22kebakaran%20hutan%22%20or%20%22kabut%20asap%22)%20min_retweets%3A10%20lang%3Aid&src=recent_search_click&f=live",
        "sec-ch-ua"                : '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        "sec-ch-ua-mobile"         : "?0",
        "sec-ch-ua-platform"       : '"macOS"',
        "sec-fetch-dest"           : "empty",
        "sec-fetch-mode"           : "cors",
        "sec-fetch-site"           : "same-origin",
        "user-agent"               : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
        "x-client-transaction-id"  : "jilpnOII1z+YE4X9QWGzgRtou+iwKaNmWwt7KS+vsvR3RIVZEjNwMxsUH/kpe7dLDQjyZ4oIFmtOx+mfYZL1FTJogZFojQ",
        "x-csrf-token"             : "812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c",
        "x-twitter-active-user"    : "yes",
        "x-twitter-auth-type"      : "OAuth2Session",
        "x-twitter-client-language": "en",
        "x-xp-forwarded-for"       : "e9a531d3f06664bbb54795a7bb835249d702383b465197259ab89c4e00ea7fc1e3f79a034b45a22bf539785b5cd9d431ff6c9ff416626a43e0f0f00ef5018aa97757900356ff138f97ce55790702d4a6a57ac63f25f8aff6cfe487aa95f90619faae9e4af66a60c5bb84d2dd344eb8fe0a4351b169a32668f0d63f89d13abfd157f0ac73235c313aa982750184128b0bede47e604e5d0b5e9b42d8b4570aa231dd87c3bbc244231ee6b19075ea76cbf4766640f7df9ce5bbe07e9ecb45a7c109207a522e2d9e40735262b3a55463efa466baa5a6beb79b19741f5dd58b37f5d7770fb143fd25d5064ae2712ebd2280abdd71caff3e490603ac152cd97077a18f6e",
        "Cookie"                   : 'guest_id_marketing=v1%3A176523474814329995; guest_id_ads=v1%3A176523474814329995; guest_id=v1%3A176523474814329995; personalization_id="v1_xw03l5DWX2lxhaNR0F3B+g=="; __cuid=32f1b2a198814f3892df122eb69af73b; g_state={"i_l":0,"i_ll":1765234768568,"i_b":"JlM/61x2AN5yK1rjGldIsiZBXDU8/YtTFzASntzP0LA"}; kdt=rLDiLi5arY92A1ZPrtq0L28MwL8L0uus7Fpp1jo5; auth_token=c2d158295830b988fe8479ca7e077d5ba9d2a9ca; ct0=812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c; lang=en; twid=u%3D1975584269155377155; __cf_bm=VnVVBWc3MU1_IPhQgrEbnTgSrKuI1mqyobEQ4Xeo_cQ-1765335030.2314074-1.0.1.1-b.oWMx1E_0Bz5gOU0CDQchdmCeygXmADTKUVMzD7FObxbwK41OmCWXJSs3sFq8vpUzfQIxIBMEIFvg5pAmxVFJLVXAn.F7CawawnGq4mWHMx.KQfmSo50HRgpXOqzm0m; __cf_bm=kwliB7cKYN.5e.Trf.hT9Cct9rIVHlQeUbAkKFLaAa4-1765334611.407966-1.0.1.1-36yJtKnuU1B8QUUOadCeQiHbFLpXonXj87LDdHMbdBU1L13I6KcYSiOb5J9CKBnSTPFU0m22GmUnkJQZbz8SDuWTKPxk3YpI19y6uJiiqty439tc8P1gGFcw3o94vYwf; guest_id=v1%3A175929449319161090; guest_id_ads=v1%3A175929449319161090; guest_id_marketing=v1%3A175929449319161090; personalization_id="v1_OGejUgP0SkhENMP/lWNnPw=="',
    },
    {
        "accept"                   : "*/*",
        "accept-language"          : "en-GB,en;q=0.9",
        "authorization"            : "Bearer AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
        "content-type"             : "application/json",
        "priority"                 : "u=1, i",
        "referer"                  : "https://x.com/search?q=(%22kebakaran%20hutan%22%20or%20%22kabut%20asap%22)%20min_retweets%3A10%20lang%3Aid&src=recent_search_click&f=live",
        "sec-ch-ua"                : '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        "sec-ch-ua-mobile"         : "?0",
        "sec-ch-ua-platform"       : '"macOS"',
        "sec-fetch-dest"           : "empty",
        "sec-fetch-mode"           : "cors",
        "sec-fetch-site"           : "same-origin",
        "user-agent"               : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
        "x-client-transaction-id"  : "/c3sEHjwA38sY4EjQQhHoqbRhM5xRKmI/l5Iq8dtugAh+tfSBxf+GsOudyOaZf0aRluBFPnVRQ61PTc/CvTmib7pThTL/g",
        "x-csrf-token"             : "812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c",
        "x-twitter-active-user"    : "yes",
        "x-twitter-auth-type"      : "OAuth2Session",
        "x-twitter-client-language": "en",
        "x-xp-forwarded-for"       : "3836daeaa8a77ebcf3eb84f644627480425c75fdc3a39389949edfe7e951eed340e634487cc7047585dfb2b191cea8e75d4d88d0c8fe16bc0618cef4a9d63b8b59a16c19a789104eecbab6b1bdc753e262193517cf10c55c3271ad76e1b5b7fdaa2f29b7a6e7cc22a43889c76bc6e176f836d460625791d60d35688312d73dc5fb2180f7c45dc30277e9a4df86e03a66c9a042de1e9651a90849df18f6555767a1acc74f74ee18db0ec05f18a6236a4b1e0f9146f386ae26c543512e08dec9e4a129dd52a7de9b3c632b31ff9e70f2e7dc2ad696ed483223e52bd1f383a392cfe4f6161c94e440ec87d6604cf93c2a3baee6dbb3a12b9d91ba4c2c64fcd748c0ba",
        "Cookie"                   : 'guest_id_marketing=v1%3A176523474814329995; guest_id_ads=v1%3A176523474814329995; guest_id=v1%3A176523474814329995; personalization_id="v1_xw03l5DWX2lxhaNR0F3B+g=="; __cuid=32f1b2a198814f3892df122eb69af73b; g_state={"i_l":0,"i_ll":1765234768568,"i_b":"JlM/61x2AN5yK1rjGldIsiZBXDU8/YtTFzASntzP0LA"}; kdt=rLDiLi5arY92A1ZPrtq0L28MwL8L0uus7Fpp1jo5; auth_token=c2d158295830b988fe8479ca7e077d5ba9d2a9ca; ct0=812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c; lang=en; twid=u%3D1975584269155377155; __cf_bm=I4oL1g96X_ofYhGpLWcOodgAg9hxFHOtA9DtNlkK95c-1765335062.234331-1.0.1.1-uOwbxoclXihtB2vyPArQGkyveLuQpsQDbPWpOMY.T52j0EIZHOYN61b5r5xiZpeOoT4cf8hppGQf3EKA.g6t3r_Rrv2cCG0miUw0WOkhjnI9blFQJQFnnK6trOn_UMMu; __cf_bm=kwliB7cKYN.5e.Trf.hT9Cct9rIVHlQeUbAkKFLaAa4-1765334611.407966-1.0.1.1-36yJtKnuU1B8QUUOadCeQiHbFLpXonXj87LDdHMbdBU1L13I6KcYSiOb5J9CKBnSTPFU0m22GmUnkJQZbz8SDuWTKPxk3YpI19y6uJiiqty439tc8P1gGFcw3o94vYwf; guest_id=v1%3A175929449319161090; guest_id_ads=v1%3A175929449319161090; guest_id_marketing=v1%3A175929449319161090; personalization_id="v1_OGejUgP0SkhENMP/lWNnPw=="',
    },
    {
        "accept"                   : "*/*",
        "accept-language"          : "en-GB,en;q=0.9",
        "authorization"            : "Bearer AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
        "content-type"             : "application/json",
        "priority"                 : "u=1, i",
        "referer"                  : "https://x.com/search?q=(%22kebakaran%20hutan%22%20or%20%22kabut%20asap%22)%20min_retweets%3A10%20lang%3Aid&src=recent_search_click&f=live",
        "sec-ch-ua"                : '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        "sec-ch-ua-mobile"         : "?0",
        "sec-ch-ua-platform"       : '"macOS"',
        "sec-fetch-dest"           : "empty",
        "sec-fetch-mode"           : "cors",
        "sec-fetch-site"           : "same-origin",
        "user-agent"               : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
        "x-client-transaction-id"  : "1dPWHL4V03GTRUTzLUOdg3ktcOf6HttQx8XoRcTRxdtVbUVhr5Kdnayq3VwD74W3XWqpPNEF701NoEN8mIxI2yHDwu9W1g",
        "x-csrf-token"             : "812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c",
        "x-twitter-active-user"    : "yes",
        "x-twitter-auth-type"      : "OAuth2Session",
        "x-twitter-client-language": "en",
        "x-xp-forwarded-for"       : "9bd3eeebaa478ec2aea19a641241d4b9fa7db0667bab892d361206f37cf7e0d9fca103fc9273918d80b102e7bc34e3874afa7013f6c4a4c042cc2f51840f01d19b27f232d8b9a6530d8aac209d7f8bed9502f630a82c21134831f2400bde3624121073e295ee8ccb8ac8f36e859c6ef6308fb55bcc6db53b10e371825e16e2d3d90a6482ee0ec18d470345f597349cfc1fb4fe8db6034a5df3adf78570284b4209dddab13d32522811a2d04726fc69863f0bafb36ffcd7cdb44c0da9a9b910e3c91ca924733b0f5f4b5680e660a4be2cd061487ee2957e759e31a72b5033bdb78e9a10c8ddb8d249d184d17665c43a73bc86a9a2c38ccd82d81b07a0c5aa1d2d69",
        "Cookie"                   : 'guest_id_marketing=v1%3A176523474814329995; guest_id_ads=v1%3A176523474814329995; guest_id=v1%3A176523474814329995; personalization_id="v1_xw03l5DWX2lxhaNR0F3B+g=="; __cuid=32f1b2a198814f3892df122eb69af73b; g_state={"i_l":0,"i_ll":1765234768568,"i_b":"JlM/61x2AN5yK1rjGldIsiZBXDU8/YtTFzASntzP0LA"}; kdt=rLDiLi5arY92A1ZPrtq0L28MwL8L0uus7Fpp1jo5; auth_token=c2d158295830b988fe8479ca7e077d5ba9d2a9ca; ct0=812911144be302cdacfffb9aa96b69f76f1febc6dbda8c9a19b08e36736607da5e1e8bf89daf98ac443cef771c8145eb037c1db1b8c379540c07d654023432d9a1522a08828467dc7ed9fd0b1cf6145c; lang=en; twid=u%3D1975584269155377155; __cf_bm=GKZ.6ffosayG5R2lIt6RwxI29HuDVHhif6xFFmMYZok-1765335087.733324-1.0.1.1-VRsKhatKAqRlKUl2eTEMfK0plCfYMZRhpz61eIog3VEfmLewKICSOU8XMCcpfzDqDraSPMx1RUDoUFk8wtXeFhVF4Ljt99tzdbjh0CiCPu_mB_S3OefmuIJm8uPeqRdD; __cf_bm=kwliB7cKYN.5e.Trf.hT9Cct9rIVHlQeUbAkKFLaAa4-1765334611.407966-1.0.1.1-36yJtKnuU1B8QUUOadCeQiHbFLpXonXj87LDdHMbdBU1L13I6KcYSiOb5J9CKBnSTPFU0m22GmUnkJQZbz8SDuWTKPxk3YpI19y6uJiiqty439tc8P1gGFcw3o94vYwf; guest_id=v1%3A175929449319161090; guest_id_ads=v1%3A175929449319161090; guest_id_marketing=v1%3A175929449319161090; personalization_id="v1_OGejUgP0SkhENMP/lWNnPw=="',
    },
]

# Scraper

In [3]:
# Initialize an empty list to store results from the scraper
scraper_result = []

In [4]:
class CircularIterator:
    def __init__(self, items):
        """Initializes the iterator with items."""
        self.items = list(items)
        self.i = 0

    def __next__(self):
        """Returns the next item, cycling back to the start if needed."""
        value = self.items[self.i % len(self.items)]
        self.i += 1
        return value

    def __iter__(self):
        """Returns the iterator itself."""
        return self

    def backward(self):
        """Moves the iterator's index back one position."""
        self.i -= 1

In [5]:
class TwitterScraper:
    """
    A scraper class for extracting data from Twitter's API.
    It handles searching timelines, fetching tweet details, and recursive scraping.
    """

    def __init__(
        self,
        search_query                   : str,
        search_timeline_url            : str,
        search_timeline_variables_param: Dict[str, Any],
        tweet_detail_url               : str,
        tweet_detail_variables_param   : Dict[str, Any],
        features_param                 : Dict[str, Any],
        iter_headers                   : CircularIterator,
        min_entries_to_scroll          : int,
    ):
        """
        Initializes the TwitterScraper with necessary API parameters and configurations.
        """
        self.search_query                    = search_query
        self.search_timeline_url             = search_timeline_url
        self.search_timeline_variables_param = search_timeline_variables_param
        self.tweet_detail_url                = tweet_detail_url
        self.tweet_detail_variables_param    = tweet_detail_variables_param
        self.features_param                  = features_param
        self.iter_headers                    = iter_headers
        self.min_entries_to_scroll           = min_entries_to_scroll

    def _generate_params(
        self, variables_param: Dict[str, Any], **variables_kwargs: Dict[str, Any]
    ) -> str:
        """
        Generates URL parameters for Twitter API requests.
        Encodes variables and features into a URL-safe string.
        """
        str_json_features  = json.dumps(self.features_param, separators=(",", ":"))
        str_json_variables = json.dumps(variables_param, separators=(",", ":"))

        # Clean up and format variables JSON string
        str_json_variables_cleaned = (
            Template(str_json_variables)
            .substitute(**{k: json.dumps(v) for k, v in variables_kwargs.items()})
            .replace('"cursor":"null"', "")
            .replace('""', '"')
            .replace(",,", ",")
        )

        quoted_features  = urllib.parse.quote(str_json_features, safe="()")
        quoted_variables = urllib.parse.quote(str_json_variables_cleaned, safe="()")

        return "variables={}&features={}".format(quoted_variables, quoted_features)

    def hit_api(
        self,
        url               : str,
        variables_param   : Dict[str, Any],
        retries           : int = 5,
        retry_delay       : int = 60,
        **variables_kwargs: Dict[str, Any],
    ) -> Dict[str, Any]:
        """
        Makes an HTTP GET request to the specified Twitter API URL.
        Includes retry logic for connection and HTTP errors.
        """
        try:
            headers  = next(self.iter_headers)  # Get next set of headers
            full_url = f"{url}?{self._generate_params(variables_param, **variables_kwargs)}"
            response = requests.get(url=full_url, headers=headers)

            response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        except requests.exceptions.ConnectionError:
            # Handle connection loss, allow user to exit or retry
            is_exit = input("Lost Connection. Want to exit? [Y/n]")
            is_exit = is_exit == "Y"

            if is_exit:
                raise

            # Recursively retry the API call
            return self.hit_api(
                url,
                variables_param,
                retries     = retries - 1,
                retry_delay = retry_delay,
                **variables_kwargs,
            )

        except requests.exceptions.HTTPError:
            # Handle HTTP errors with retries
            if retries > 0:
                time.sleep(retry_delay)

                # Special handling for specific URLs on error (e.g., reset cursor)
                if url == self.search_timeline_url:
                    self.hit_api(
                        self.tweet_detail_url,
                        self.tweet_detail_variables_param,
                        retries  = 0,
                        cursor   = None,
                        tweet_id = 1,
                    )

                else:
                    self.hit_api(
                        self.search_timeline_url,
                        self.search_timeline_variables_param,
                        retries = 0,
                        cursor  = None,
                        query   = "",
                    )

                # Recursively retry the API call
                return self.hit_api(
                    url,
                    variables_param,
                    retries     = retries - 1,
                    retry_delay = retry_delay,
                    **variables_kwargs,
                )

            else:
                raise  # Re-raise if no retries left

        return response.json()

    def stream_id_from_timeline(self, max_scroll: int) -> Generator[int, None, None]:
        """
        Streams tweet IDs from the search timeline based on the search query.
        Scrolls through pages until max_scroll or no more entries.
        """
        cursor = None

        for _ in range(max_scroll):
            try:
                response_json = self.hit_api(
                    self.search_timeline_url,
                    self.search_timeline_variables_param,
                    query  = self.search_query,
                    cursor = cursor,
                )

            except requests.exceptions.HTTPError as e:
                print(f"Error fetching ids from timeline: {self.search_query}. error: {e}")
                break

            # Extract instructions from the API response
            instructions = (
                response_json.get("data", {})
                .get("search_by_raw_query", {})
                .get("search_timeline", {})
                .get("timeline", {})
                .get("instructions", [])
            )

            entries = []
            for instruction in instructions:
                if "entries" in instruction:
                    entries.extend(instruction.get("entries", []))
                elif "entry" in instruction:
                    entries.append(instruction.get("entry", {}))

            is_empty = True
            for entry in entries:
                entry_id = entry.get("entryId")

                if "tweet" in entry_id:
                    is_empty = False

                    # Extract tweet ID
                    tweet_id = (
                        entry.get("content", {})
                        .get("itemContent", {})
                        .get("tweet_results", {})
                        .get("result", {})
                        .get("rest_id")
                    )

                    if tweet_id:
                        yield tweet_id  # Yield the tweet ID

                elif "cursor" in entry_id:
                    cursor = entry.get("content", {}).get("value")  # Update cursor for next page

            # Break if no new entries or minimum scroll threshold not met
            if is_empty or len(entries) < self.min_entries_to_scroll:
                break

    def stream_tweet_detail(
        self, parent_tweet_id: str, max_scroll: int
    ) -> Generator[Tuple[str, str, str], None, None]:
        """
        Streams detailed information for a given tweet ID, including replies/conversation.
        Scrolls through pages until max_scroll or no more entries.
        """
        cursor = None

        for _ in range(max_scroll):
            self.iter_headers.backward()  # Move to previous header set

            try:
                response_json = self.hit_api(
                    self.tweet_detail_url,
                    self.tweet_detail_variables_param,
                    tweet_id = parent_tweet_id,
                    cursor   = cursor,
                )

            except requests.exceptions.HTTPError as e:
                print(f"Error fetching tweet detail: {parent_tweet_id}. error: {e}")
                break

            # Extract instructions from the API response
            instructions = (
                response_json.get("data", {})
                .get("threaded_conversation_with_injections_v2", {})
                .get("instructions", [])
            )

            entries = []
            for instruction in instructions:
                if "entries" in instruction:
                    entries.extend(instruction.get("entries", []))
                elif "entry" in instruction:
                    entries.append(instruction.get("entry", {}))

            is_empty = True
            for entry in entries:
                entry_id = entry.get("entryId")

                if "tweet" in entry_id or "conversationthread" in entry_id:
                    is_empty = False
                    content = entry.get("content", {})

                    if "items" in content:
                        content = content.get("items")[0].get("item")

                    content_result = (
                        content.get("itemContent", {})
                        .get("tweet_results", {})
                        .get("result", {})
                    )

                    # Extract tweet details
                    tweet_id = content_result.get("rest_id")
                    username = (
                        content_result.get("core", {})
                        .get("user_results", {})
                        .get("result", {})
                        .get("core", {})
                        .get("screen_name")
                    )
                    tweet_text = content_result.get("legacy", {}).get("full_text")
                    retweet_count = content_result.get("legacy", {}).get("retweet_count")

                    if tweet_id:
                        yield tweet_id, username, tweet_text, retweet_count

                elif "cursor" in entry_id:
                    cursor = entry.get("content", {}).get("value")  # Update cursor for next page

            # Break if no new entries or minimum scroll threshold not met
            if is_empty or len(entries) < self.min_entries_to_scroll:
                break

    def scrape_to_storage(self, storage: list, max_scroll: int, max_depth: int, progress_bar: tqdm):
        """
        Scrapes tweets and their conversations, storing results in a list.
        Supports recursive scraping for quoted tweets up to a specified depth.
        """
        gen_id_from_timeline = self.stream_id_from_timeline(max_scroll)
        for parent_tweet_id in gen_id_from_timeline:
            progress_bar.set_description(f"Tweet ID: {parent_tweet_id}")

            parent_retweet_count = 0

            gen_tweet_detail = self.stream_tweet_detail(parent_tweet_id, max_scroll)
            for tweet_id, username, tweet_text, retweet_count in gen_tweet_detail:
                if tweet_id == parent_tweet_id:
                    parent_retweet_count = retweet_count

                # Store extracted tweet data
                storage.append(
                    {
                        "query"          : self.search_query,
                        "parent_tweet_id": parent_tweet_id,
                        "tweet_id"       : tweet_id,
                        "username"       : username,
                        "tweet_text"     : tweet_text,
                    }
                )

                progress_bar.update(1)

            current_max_depth = max_depth - 1
            # Recursively scrape quoted tweets if depth allows and retweets exist
            if current_max_depth > 0 and parent_retweet_count > 0:
                child_scraper = TwitterScraper(
                    search_query                    = f"quoted_tweet_id:{parent_tweet_id}",
                    search_timeline_url             = self.search_timeline_url,
                    search_timeline_variables_param = self.search_timeline_variables_param,
                    tweet_detail_url                = self.tweet_detail_url,
                    tweet_detail_variables_param    = self.tweet_detail_variables_param,
                    features_param                  = self.features_param,
                    iter_headers                    = self.iter_headers,
                    min_entries_to_scroll           = self.min_entries_to_scroll,
                )

                # Call scrape_to_storage recursively for child tweets
                child_scraper.scrape_to_storage(storage, max_scroll, current_max_depth, progress_bar)

                progress_bar.update(1)

In [6]:
# Initialize the TwitterScraper with API configurations
scraper = TwitterScraper(
    search_query                    = API_SEARCH_QUERY,
    search_timeline_url             = API_SEARCH_TIMELINE_URL,
    search_timeline_variables_param = API_SEARCH_TIMELINE_VARIABLES_PARAM,
    tweet_detail_url                = API_TWEET_DETAIL_URL,
    tweet_detail_variables_param    = API_TWEET_DETAIL_VARIABLES_PARAM,
    features_param                  = API_FEATURES_PARAM,
    iter_headers                    = CircularIterator(API_LIST_HEADERS),
    min_entries_to_scroll           = 5,
)

In [7]:
# Scrape data and store it in the specified storage
scraper.scrape_to_storage(
    storage      = scraper_result,
    max_scroll   = API_MAX_SCROLL,
    max_depth    = API_MAX_DEPTH,
    progress_bar = tqdm(),
)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [8]:
scraper_result

[]

In [9]:
len(scraper_result)

0

In [10]:
# Save the scraped tweets to a JSON file
with open("tweets.json", "w") as f:
    json.dump(scraper_result, f, indent=4)